<a href="https://colab.research.google.com/github/ChloeReads/Software-Engineering-Pipeline/blob/main/Synthetic%20Data%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import files
uploaded = files.upload()

Saving requirements.txt to requirements.txt


In [28]:
pip install -r requirements.txt

In [40]:
import random
import pandas as pd
from faker import Faker
fake = Faker('en_GB')

In [55]:
locations = ['Edinburgh', 'Glasgow', ' Aberdeen', 'Leeds', 'Manchester', 'London', 'Bristol', 'Cardiff', 'Belfast', 'Birmingham', 'Brighton']
departments = ['Human Resources', 'Risk Management','Sales','Marketing','Adminstration','Engineering','Investment','Sciences']

In [63]:
data = []
for i in range(10000):
    new_row = {
        'ID': fake.random_int(min=10000, max=99999),
        'Full Name': fake.name(),
        'Location': fake.random_element(elements=locations),
        'Department': fake.random_element(elements=departments) # Use the predefined list of departments
    }
    data.append(new_row)

    # Occasionally duplicate a row (e.g., 5% of the time)
    if random.random() < 0.05 and len(data) > 0:
        row_to_duplicate = random.choice(data)
        data.append(row_to_duplicate)


df_staff = pd.DataFrame(data)
display(df_staff.head())

,ID,Full Name,Location,Department
0,42962,Steven Watson,Edinburgh,Adminstration
1,99703,Tanya Ryan,Belfast,Human Resources
2,86354,Jennifer Miller,Leeds,Investment
3,93505,Tammy Davis,Aberdeen,Engineering
4,77026,Ralph Lin,Belfast,Human Resources


In [64]:
# Extract unique IDs and their departments from the existing DataFrame
df_teams_data = df[['ID', 'Department']].drop_duplicates().copy()

# Generate a 'Team' for each Department. Since we want a team structure *within* departments,
# let's create a few teams per department.
teams_per_department = {}
for department in df_teams_data['Department'].unique():
    teams_per_department[department] = [f'{department} Team {i+1}' for i in range(random.randint(1, 4))] # 1 to 4 teams per department

# Assign a random team from the department's teams to each ID-Department pair
df_teams_data['Team'] = df_teams_data['Department'].apply(lambda x: random.choice(teams_per_department[x]))

# Now we can merge this back with the original df if needed, or just use this new df for teams
# For this request, I'll just display the new teams dataset
df_teams = df_teams_data
display(df_teams.head())

,ID,Department,Team
0,42962,Adminstration,Adminstration Team 1
1,99703,Human Resources,Human Resources Team 2
2,86354,Investment,Investment Team 1
3,93505,Engineering,Engineering Team 1
4,77026,Human Resources,Human Resources Team 2


In [65]:
# Get the IDs from the original DataFrame
ids = df['ID'].unique().tolist()

# Generate random pay rates for each ID
pay_rates_data = []
for id in ids:
    pay_rates_data.append({
        'ID': id,
        'Pay Rate': round(fake.random_number(digits=2) + fake.pydecimal(left_digits=2, right_digits=2, positive=True), 2)
    })

df_pay_rates = pd.DataFrame(pay_rates_data)
display(df_pay_rates.head())

,ID,Pay Rate
0,42962,63.69
1,99703,84.76
2,86354,134.80
3,93505,117.14
4,77026,102.47


In [66]:
df_staff.to_csv('Staff.csv')
df_teams.to_csv('Teams.csv')
df_pay_rates.to_csv('PayRates.csv')